In [ ]:
# =======================
# 📌 Step 1: Install build tools
# =======================
!apt update
!apt install -y python3-pip zip unzip openjdk-17-jdk
!pip install buildozer cython==0.29.36 kivy kivymd pillow

# =======================
# 📌 Step 2: Upload your mines-detector.zip
# =======================
from google.colab import files
uploaded = files.upload()

!unzip mines-detector.zip -d mines_app

# =======================
# 📌 Step 3: Replace UI with Mobile-Friendly KivyMD version
# =======================
import os

main_py = """
from kivy.lang import Builder
from kivymd.app import MDApp
from kivymd.uix.filemanager import MDFileManager
from kivymd.uix.image import FitImage
from kivy.uix.boxlayout import BoxLayout
from kivy.core.window import Window
from kivy.clock import mainthread
from PIL import Image
import joblib
import numpy as np

Window.size = (360, 640)  # For testing in Colab preview

KV = '''
BoxLayout:
    orientation: "vertical"
    MDTopAppBar:
        title: "Mines Detector"
    MDRaisedButton:
        text: "Upload Screenshot"
        pos_hint: {"center_x": 0.5}
        on_release: app.file_manager_open()
    FitImage:
        id: img_preview
        source: ""
    MDRaisedButton:
        text: "Detect Mines"
        pos_hint: {"center_x": 0.5}
        on_release: app.detect_mines()
'''

class MainApp(MDApp):
    def build(self):
        self.file_manager = MDFileManager(
            exit_manager=self.exit_manager,
            select_path=self.select_path
        )
        return Builder.load_string(KV)

    def file_manager_open(self):
        self.file_manager.show('/content')  # Change path if needed

    def exit_manager(self, *args):
        self.file_manager.close()

    @mainthread
    def select_path(self, path):
        self.image_path = path
        self.root.ids.img_preview.source = path
        self.exit_manager()

    def detect_mines(self):
        model = joblib.load("mines_app/mines_kmeans.pkl")
        img = Image.open(self.image_path).convert("RGB").resize((16, 16))
        features = np.array(img).reshape(-1, 3)
        preds = model.predict(features)
        self.root.ids.img_preview.source = self.image_path
        print("Prediction:", preds.reshape(16, 16))

MainApp().run()
"""

with open("mines_app/main.py", "w") as f:
    f.write(main_py)

# =======================
# 📌 Step 4: Buildozer Spec
# =======================
spec = """
[app]
title = Mines Detector
package.name = minesdetector
package.domain = org.minesdetector
source.dir = .
source.include_exts = py,png,jpg,kv,atlas,pkl
icon.filename = %(source.dir)s/icon.png
version = 1.0
requirements = python3,kivy,kivymd,pillow,joblib
orientation = portrait
fullscreen = 0

[buildozer]
log_level = 2
warn_on_root = 1
android.api = 34
android.minapi = 21
android.ndk = 25b
android.arch = armeabi-v7a
android.sdk_path = /content/.buildozer/android/platform/android-sdk
android.ndk_path = /content/.buildozer/android/platform/android-ndk-r25b
"""

with open("mines_app/buildozer.spec", "w") as f:
    f.write(spec)

# =======================
# 📌 Step 5: Build APK
# =======================
%cd mines_app
!buildozer -v android debug

# =======================
# 📌 Step 6: Download APK
# =======================
from google.colab import files
files.download("bin/minesdetector-1.0-debug.apk")
